In [1]:
# Imports

import apache_beam as beam

Failed to import GCSFileSystem; loading of this filesystem will be skipped. Error details: cannot import name 'storage' from 'google.cloud' (unknown location)


In [ ]:
# Flights

# Auxiliar Class
class filter_values(beam.DoFn):
    def process(self, record):
        if int(record[8]) > 0:
            return [record]

# Pipeline Define
pl1 = beam.Pipeline()

pcolection_sum_delays = (
    pl1
    | "Data Reading (1)" >> beam.io.ReadFromText("../data/voos_sample.csv", skip_header_lines = 1)
    | "Split by Common (1)" >> beam.Map(lambda record: record.split(','))
#    | "Filter by Delay (1)" >> beam.Filter(lambda record: int(record[8])>0)
    | "Filter by Delay (1)" >> beam.ParDo(filter_values())
    | "Create Pair (1)" >> beam.Map(lambda record: (record[4], int(record[8])))
    | "Sum by Key" >> beam.CombinePerKey(sum)
#    | "Print Result (1)" >> beam.Map(print)
#    | "Save Result" >> beam.io.WriteToText("../data/voos.txt")
)

pcolection_count_delays = (
    pl1
    | "Data Reading (2)" >> beam.io.ReadFromText("../data/voos_sample.csv", skip_header_lines = 1)
    | "Split by Common (2)" >> beam.Map(lambda record: record.split(','))
#    | "Filter by Delay (2)" >> beam.Filter(lambda record: int(record[8])>0)
    | "Filter by Delay (2)" >> beam.ParDo(filter_values())
    | "Create Pair (2)" >> beam.Map(lambda record: (record[4], int(record[8])))
    | "Count by Key" >> beam.combiners.Count.PerKey()
#    | "Print Result (2)" >> beam.Map(print)
#    | "Save Result" >> beam.io.WriteToText("../data/voos.txt")
)

delay_table = (
    {"Count": pcolection_count_delays, "Sum": pcolection_sum_delays}
    | "Group By" >> beam.CoGroupByKey()
    | "Print Result" >> beam.Map(print)
)

# Executation
pl1.run()

('LAX', {'Count': [4], 'Sum': [92]})
('HNL', {'Count': [1], 'Sum': [15]})
('DFW', {'Count': [1], 'Sum': [95]})
('OGG', {'Count': [1], 'Sum': [138]})
('JFK', {'Count': [4], 'Sum': [220]})


In [3]:
# Poem

words = ["quatro", "um"]

# Auxiliar Functions
def find_word(record):
    return record in words

# Pipeline Define
pl2 = beam.Pipeline()

pcolection = (
    pl2
    | "Data Reading" >> beam.io.ReadFromText("../data/poema.txt")
    | "Split by Common" >> beam.FlatMap(lambda record: record.split(" "))
    | "Filter by Function" >> beam.Filter(find_word)
    | "Print Result" >> beam.Map(print)
)

pl2.run()


quatro
quatro
um
quatro
quatro
um


In [4]:
# Ethnicity

# Pipeline Define
pl3 = beam.Pipeline()

blacks = ("Adão", "Jesus", "Mike")
whites = ("Tulio", "Mary", "Joca")
indians = ("Vic", "Marta", "Tom")

black_pc = pl3 | "Create a pcollection to blacks" >> beam.Create(blacks)
whites_pc = pl3 | "Create a pcollection to whites" >> beam.Create(whites)
indians_pc = pl3 | "Create a pcollection to indians" >> beam.Create(indians)

people = (
    (black_pc, whites_pc, indians_pc)
    | beam.Flatten()
    | beam.Map(print)
)

pl3.run()

Vic
Marta
Tom
Tulio
Mary
Joca
Adão
Jesus
Mike
